In [ ]:
!pip install opencv-python scikit-image

In [ ]:
import kagglehub
path = kagglehub.dataset_download('dataclusterlabs/indian-vehicle-dataset')
print("Path to dataset files:", path)
print('Data source import complete.')

100%|██████████| 2.37G/2.37G [00:25<00:00, 99.3MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/dataclusterlabs/indian-vehicle-dataset/versions/4
Data source import complete.


In [ ]:
import os
import cv2
import numpy as np
from skimage.feature import hog
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Set path to dataset
data_dir = path+'/Vehicle_5_classes_sample'  # adjust if needed

X = []
y = []

# Resize image to a fixed size
IMG_SIZE = (224, 224)

# Loop through folders and extract HOG features
for class_name in os.listdir(data_dir):
    class_dir = os.path.join(data_dir, class_name)
    if os.path.isdir(class_dir):
        for file in os.listdir(class_dir):
            file_path = os.path.join(class_dir, file)
            img = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)
            if img is not None:
                img = cv2.resize(img, IMG_SIZE)
                features, _ = hog(img, orientations=9, pixels_per_cell=(8, 8),
                                  cells_per_block=(2, 2), visualize=True, block_norm='L2-Hys')
                X.append(features)
                y.append(class_name)

X = np.array(X)
y = np.array(y)

# Encode labels
le = LabelEncoder()
y = le.fit_transform(y)

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

params = {
    'C': [0.1, 1, 10],
    'kernel': ['rbf', 'linear'],
    'gamma': ['scale', 'auto']
}

grid = GridSearchCV(SVC(), params, cv=3)
grid.fit(X_train_scaled, y_train)

print("Best Parameters:", grid.best_params_)
y_pred = grid.predict(X_test_scaled)


Best Parameters: {'C': 10, 'gamma': 'scale', 'kernel': 'rbf'}


In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

# Train an SVM model
svm = SVC(kernel='linear', C=1.0)
svm.fit(X_train, y_train)

# Make predictions
y_pred = svm.predict(X_test)

# Evaluate model
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred, target_names=le.classes_))


Accuracy: 0.31543624161073824

Classification Report:
                  precision    recall  f1-score   support

           Auto       0.17      0.12      0.14        33
            Bus       0.16      0.14      0.15        28
Tempo Traveller       0.36      0.33      0.35        24
        Tractor       0.46      0.39      0.42        31
          Truck       0.36      0.58      0.44        33

       accuracy                           0.32       149
      macro avg       0.30      0.31      0.30       149
   weighted avg       0.30      0.32      0.30       149



In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=100)
rf.fit(X_train, y_train)
rf_pred = rf.predict(X_test)

print("Random Forest Accuracy:", accuracy_score(y_test, rf_pred))
